# Prediction

input: zmatrix file location, dimensions , number of users, number of topics, topn_topics
<br> output: userId, topic id (topn topicId for each user)

<br> STEPS
<br> 1. Extracting zmatrix weights (sometimes all dimensions  are not filled in, hence add zero where a dimension value is not present
<br> 2. Calculate z_matrix * transpose(z_matrix)
<br> 3. Extract values from users(rows) and topic(0 to 100 columns)
<br> 4. Write the results into file


In [1]:
import numpy as np
import pandas as pd
import re
np.set_printoptions(suppress=True) 
import shutil

In [2]:
#please manually make sure all dimension values for all nodes, otherwise it will throw error

def link_prediction(file_location, dimension, number_of_users, number_of_topics,topn):
    #create an empty array for zmatrix
    total_nodes = number_of_users + number_of_topics
    zmatrix=np.zeros((total_nodes,dimension))
    #read the file
    f = open(file_location, "r")
    f.readline()
    i=0
    for line in f:
        line = line.strip()
        row_array = line.split(':')
        
        #skip the first element of row_array
#         str1 = ''.join(row_array[0])
#         node, num_of_nodes = str1.split(',')
        del row_array[0]
        #each element in a line
        weight=np.zeros(dimension)
  
        for index, pair in enumerate(row_array): 
            x1,x2 = pair.split(",")
            #print(type(index),type(x1), type(x2))
            x1 = int(x1)
            x2 = float(x2)
            #print(index,x1,x2)
            #take only the second value, which is weight
            #append the weight
            weight[x1]=x2
        #after complting one row, add to zmatrix    
        zmatrix[i] = weight
        i=i+1
    f.close()
    
    #print(zmatrix)
    np.savetxt('updated_zmatrix', zmatrix, delimiter=',')   # X is an array
    
    #calculate dot product
    #get the transpose
    zmatrix_t = np.transpose(zmatrix)
    #dot product 
    result = np.dot(zmatrix,zmatrix_t)
    
    #save the result to file
    folder, file = file_location.split('/')
    file_zproduct =  folder + '/zproduct.csv'
    file_ptr = open(file_zproduct, 'w')
    np.savetxt(file_ptr, result, fmt='%10.10f', delimiter="\t")
    file_ptr.close()
    
    #read the results from zproduct.csv
    names=['UserId', 'TopicId', 'Weight']
    headers = np.arange(0,total_nodes,1)
    df_result = pd.read_csv(file_zproduct, sep='\t', names=headers) 
    
    #print(df_result)
       
    #read only users. For example stats from 100
    df_result = df_result.iloc[number_of_topics:number_of_topics + number_of_users,:]
    
    #consume only first k columns as topics
    df_result = df_result.iloc[:,0:number_of_topics]
    
    #skipped the normalised as we are focused on top k highest weight than actual prediction of weights
    #find n largest weights and their index 
    #delete the content if file_name already exists
    file_predicted = folder + '/predicted.csv'
    open(file_predicted, 'w').close()
    for index, row_user in df_result.iterrows():
        #print((row_user.nlargest(5)))
        #print(row_user)
        top_n = row_user.nlargest(topn)
        topics= top_n.index
        weights=top_n.values

        # create a dataframe with these values
        topn_userinterest=pd.DataFrame({'UserId': np.repeat(index, topn), 'TopicId':topics,'Weight':weights})
        topn_userinterest.to_csv(file_predicted, encoding='utf-8', index=False, sep='\t', mode='a', header=False)
        #print(topn_userinterest)


In [48]:
# def link_prediction(file_location, dimension, number_of_users, number_of_topics,topn):
print("Predicted topk topics, Please wait...")
link_prediction("Z_All_t4_c100/Zmatrix59", 100,2458,100,10)
print("Done")

Predicted topk topics, Please wait...
Done
